### 샌드위치 가게의 세부 메뉴를 설명하는 기사의 정보를 가져오기

In [2]:
import requests
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
headers = {'User-Agent': 'Mozilla/5.0'}

import pandas as pd

In [3]:
df = pd.read_csv('data/best_sandwiches_list_chicago.csv', index_col= 0)
df.head()

,Rank,Cafe,Menu,URL
0,1,Old Oak Tap,BLT,https://www.chicagomag.com/Chicago-Magazine/No...
1,2,Au Cheval,Fried Bologna,https://www.chicagomag.com/Chicago-Magazine/No...
2,3,Xoco,Woodland Mushroom,https://www.chicagomag.com/Chicago-Magazine/No...
3,4,Al’s Deli,Roast Beef,https://www.chicagomag.com/Chicago-Magazine/No...
4,5,Publican Quality Meats,PB&L,https://www.chicagomag.com/Chicago-Magazine/No...


### URL 컬럼에 있는 내용을 50개 읽어서 각 페이지에서 가게 주소, 대표 샌드위치 가격, 가게 전화번호 추출

In [4]:
df['URL'][0]

'https://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Old-Oak-Tap-BLT/'

In [5]:
html = requests.get(df['URL'][0], headers=headers)
soup_tmp = BeautifulSoup(html.text)
soup_tmp

<!DOCTYPE html>
<html lang="en-US">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<link href="https://gmpg.org/xfn/11" rel="profile"/>
<title>1. Old Oak Tap BLT – Chicago Magazine</title>
<style type="text/css">			.heateorSssInstagramBackground{background:radial-gradient(circle at 30% 107%,#fdf497 0,#fdf497 5%,#fd5949 45%,#d6249f 60%,#285aeb 90%)}
						div.heateor_sss_horizontal_sharing i.heateorSssInstagramBackground{background:#000!important;}div.heateor_sss_standard_follow_icons_container i.heateorSssInstagramBackground{background:#000;}
										.heateor_sss_horizontal_sharing .heateorSssSharing,.heateor_sss_standard_follow_icons_container .heateorSssSharing{
							background-color: #000;
							color: #fff;
						border-width: 0px;
			border-style: solid;
			border-color: transparent;
		}
				.heateor_sss_horizontal_sharing .heateorSssTCBackground{
			color:#666;
		}
				.heateor_sss_horizontal_sharing .heateorSssSharing:hover,.heateor_ss

#### 위 URL에서 확인한 주소, 가격 전화번호 정보

In [6]:
print(soup_tmp.find('p', 'addy'))

<p class="addy">
<em>$10. 2109 W. Chicago Ave., 773-772-0406, <a href="http://www.theoldoaktap.com/">theoldoaktap.com</a></em></p>


#### 텍스트만 추출하고 빈칸으로 나누기

In [7]:
price_tmp = soup_tmp.find('p', 'addy').get_text()
price_tmp

'\n$10. 2109 W. Chicago Ave., 773-772-0406, theoldoaktap.com'

In [8]:
price_tmp.split()

['$10.', '2109', 'W.', 'Chicago', 'Ave.,', '773-772-0406,', 'theoldoaktap.com']

#### 가격정보

In [9]:
price_tmp.split()[0]

'$10.'

#### 가격정보 뒤의 . 슬라이싱하기

In [10]:
price_tmp.split()[0][:-1]

'$10'

#### 가격정보를 제외하고 한 문장으로 만들어 주소 추출하기

In [11]:
' '.join(price_tmp.split()[1:-2])

'2109 W. Chicago Ave.,'

### for문을 이용하여 반복문 돌리기

In [28]:
price = []
address = []

for n in df.index[:5]:
    html = requests.get(df['URL'][n], headers=headers)
    soup_tmp = BeautifulSoup(html.text)
    
    gettings = soup_tmp.find('p', 'addy').get_text()
    
    price.append(gettings.split()[0][:-1])
    address.append(' '.join(gettings.split()[1:-2]))

In [23]:
price

['$10', '$9', '$9.50', '$9.40', '$10']

In [24]:
address

['2109 W. Chicago Ave.,',
 '800 W. Randolph St.,',
 '445 N. Clark St.,',
 '914 Noyes St., Evanston,',
 '825 W. Fulton Mkt.,']

### 상태 진행바까지 적용하고 다시 샌드위치 페이지 50개에 접근하기
### 5개에서 Failed to establish a new connection: [Errno 11001] getaddrinfo failed')) 에러 발생

In [15]:
from tqdm import tqdm_notebook
import time

price = []
address = []

for n in tqdm_notebook(df.index):
    html = requests.get(df['URL'][n], headers=headers)
    soup_tmp = BeautifulSoup(html.text, 'lxml')
    
    gettings = soup_tmp.find('p', 'addy').get_text()
    
    price.append(gettings.split()[0][:-1])
    address.append(' '.join(gettings.split()[1:-2]))


C:\Users\Public\Documents\ESTsoft\CreatorTemp/ipykernel_21716/989462874.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for n in tqdm_notebook(df.index):


  0%|          | 0/50 [00:00<?, ?it/s]

ConnectionError: HTTPSConnectionPool(host='www.chicagomag.comhttps', port=443): Max retries exceeded with url: //www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Hendrickx-Belgian-Bread-Crafter-Belgian-Chicken-Curry-Salad/ (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000002AC1A205730>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

In [25]:
price

['$10', '$9', '$9.50', '$9.40', '$10']

In [26]:
address

['2109 W. Chicago Ave.,',
 '800 W. Randolph St.,',
 '445 N. Clark St.,',
 '914 Noyes St., Evanston,',
 '825 W. Fulton Mkt.,']

In [32]:
df_5 = df.head(5)

In [38]:
df_5['Price'] = price
df_5['Address'] = address

df_5 = df_5.loc[:, ['Rank', 'Cafe', 'Menu', 'Price', 'Address']]
df_5.set_index('Rank', inplace = True)
df_5.head()

C:\Users\Public\Documents\ESTsoft\CreatorTemp/ipykernel_21716/2348670822.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_5['Price'] = price
C:\Users\Public\Documents\ESTsoft\CreatorTemp/ipykernel_21716/2348670822.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_5['Address'] = address


,Cafe,Menu,Price,Address
Rank,,,,
1,Old Oak Tap,BLT,$10,"2109 W. Chicago Ave.,"
2,Au Cheval,Fried Bologna,$9,"800 W. Randolph St.,"
3,Xoco,Woodland Mushroom,$9.50,"445 N. Clark St.,"
4,Al’s Deli,Roast Beef,$9.40,"914 Noyes St., Evanston,"
5,Publican Quality Meats,PB&L,$10,"825 W. Fulton Mkt.,"


In [39]:
df_5.to_csv('data/best_sandwiches_list_chicago2.csv', sep =',',
           encoding = 'utf-8')